In [ ]:
import os,sys
from slr.model.st_gcn import st_gcn
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import time
from tqdm import tqdm
from slr.data.datagenerator import TFRecDataGenerator
os.getcwd()


In [2]:
from slr.data.ksl.ksl_adj import Graph

In [6]:

dataset = TFRecDataGenerator("test_with_preprocess.tfrec",batch_size=16)

In [6]:
model = st_gcn.Model(channel = 3,
            num_class=100,
            window_size=150,
            num_point=137,
            graph = Graph('spatial')).cuda(0)

NameError: name 'Graph' is not defined

In [4]:
# from torchsummary import summary
# summary(model, (3,150,137,1),device='cuda')

In [11]:
from slr.data import DataPath
from slr.data.datagenerator import KeyDataGenerator
from sklearn.model_selection import train_test_split
bat_size=8
class_limit=100

x_train, x_test, y_train, y_test = DataPath(class_limit).split_data

train_generator = KeyDataGenerator(x_train,y_train,batch_size = bat_size,scale=True,seq_len=150)
test_generator = KeyDataGenerator(x_test,y_test,batch_size = bat_size, seq_len=150)

In [ ]:
train_generator[0][0].reshape((bat_size,-1,137,3))
train_generator[0][1]

In [5]:
from slr.model.trainer import TorchTrainer
criterion = nn.CrossEntropyLoss().float().cuda(0)
optimizer = optim.Adam

device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
trainer = TorchTrainer(model,
            epochs = 200,
            train_loader = train_generator,
            test_loader = test_generator,
            optim= optimizer,
            criterion = criterion,
            name = 'st_gen trainer',
            device = device,
            )


NameError: name 'model' is not defined